In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.4 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 


In [ ]:
# thu vien chinh sua cac loi unicode
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
import string    
import random
import json
import spacy
from spacy import displacy
from transformers import BertTokenizer, BertForTokenClassification
from transformers import TFRobertaModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-07 17:11:07--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-07 17:11:08--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-07 17:11:08--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import json
import os
import shutil

In [ ]:
spark = sparknlp.start()

In [ ]:
# connect ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#save_model_address = '/content/drive/MyDrive/LUẬN VĂN-K18_CQ/02. Bình-Ngọc/Data/Dataset/conll/b_simp'
#save_model_address = '/content'

save_model_address = '/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config'

#save_model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=20)
#tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=True)

model = AutoModelForTokenClassification.from_pretrained(save_model_address)
tokenizer = AutoTokenizer.from_pretrained(save_model_address, do_lower_case=True, model_max_length=256)

#nlp = pipeline("ner", model=save_model, tokenizer=tokenizer, aggregation_strategy='simple',ignore_labels =['X','O'])


In [ ]:
loaded_ner_model = NerDLModel.load("/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config/pytorch_model")

In [ ]:
df = spark.read.option('header','true').csv('/content/drive/MyDrive/Data Science/thesis/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)

In [ ]:
df

DataFrame[name: string, link: string, rating: string, enroll: string, instructor: string, time: string, levelrequirement: string, skillrequirement: string, SkillWillLearn: string, SkillGain: string, Subject: string, organization: string, fee: string, program: string, RelationInsOrg: string, Subtitle: string]

In [ ]:
import pyspark.sql.functions as f
df = df.withColumn("enroll", f.regexp_replace(f.col("enroll"), ",", "").alias("enroll")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hours", "").alias("time")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hour", "").alias("time")) \
    .withColumn("fee", f.regexp_replace(f.col("fee"), "[a-zA-Z]+", "").alias("fee"))
    #.withColumn("fee", f.regexp_replace(f.col("fee"), "Enroll", "").alias("fee"))

In [ ]:
from pyspark.sql.types import FloatType, LongType, IntegerType

df = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("enroll",
              df["enroll"]
              .cast(LongType())) \
  .withColumn("time",df['time'].cast(IntegerType())) \
  .withColumn("fee", df['fee'].cast(IntegerType()))

In [ ]:
df

DataFrame[name: string, link: string, rating: float, enroll: bigint, instructor: string, time: int, levelrequirement: string, skillrequirement: string, SkillWillLearn: string, SkillGain: string, Subject: string, organization: string, fee: int, program: string, RelationInsOrg: string, Subtitle: string]

In [ ]:
df.show()

+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|                name|                link|rating| enroll|          instructor|time|levelrequirement|    skillrequirement|      SkillWillLearn|           SkillGain|             Subject|        organization| fee|             program|      RelationInsOrg|            Subtitle|
+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|    Machine Learning|https://www.cours...|   4.9|4491564|           Andrew Ng|  61|            null|                null|Machine learning ...|Logistic Regressi...|        Dat

In [ ]:
from pyspark.sql.functions import col,when

In [ ]:
df = df.na.fill(value=0, subset=['rating','enroll', 'fee'])

In [ ]:
df = df.na.fill(value='None', subset=['link','instructor', 'levelrequirement', 'skillrequirement', 'SkillWillLearn','SkillGain', 'Subject', 'organization', 'program', 'RelationInsOrg', 'Subtitle'])

In [ ]:
df = df.withColumn("levelrequirement", f.regexp_replace(f.col("levelrequirement"), "None", "Beginener").alias("levelrequirement")) \
    .withColumn("levelrequirement", f.regexp_replace(f.col("levelrequirement"), "Advanced", "Expert").alias("levelrequirement")) 


In [ ]:
df.show()

+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+
|                name|                link|rating| enroll|          instructor|time|levelrequirement|    skillrequirement|      SkillWillLearn|           SkillGain|             Subject|        organization|fee|             program|      RelationInsOrg|            Subtitle|
+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+
|    Machine Learning|https://www.cours...|   4.9|4491564|           Andrew Ng|  61|       Beginener|                None|Machine learning ...|Logistic Regressi...|        Data S

In [ ]:
from pyspark.sql.functions import split, col

df2 = df.select(col('name'),col('link'),col('rating'),col('enroll'),col('instructor'),col('time'),col('levelrequirement'),col('skillrequirement'),
                col('SkillWillLearn'),col('SkillGain'),col('Subject'),col('organization'),col('fee'),col('program'),col('RelationInsOrg'),split(col("Subtitle"),", ").alias("Subtitle"))
df2.show()

+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+
|                name|                link|rating| enroll|          instructor|time|levelrequirement|    skillrequirement|      SkillWillLearn|           SkillGain|             Subject|        organization|fee|             program|      RelationInsOrg|            Subtitle|
+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+
|    Machine Learning|https://www.cours...|   4.9|4491564|           Andrew Ng|  61|       Beginener|                None|Machine learning ...|Logistic Regressi...|        Data S

In [ ]:
!pip install dbutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import dbutils

dbutils.widgets.text("url", "neo4j+s://c4ae1994.databases.neo4j.io")
dbutils.widgets.text("user", "neo4j")
dbutils.widgets.text("user", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")

AttributeError: ignored

In [ ]:
df = spark.read.csv("Cases Report.csv")
df = spark.read.option('header','true').csv('Cases Report.csv',inferSchema=True)
#.option("query","merge (course:Course{courseIdentifyKey: apoc.util.sha512([]) , courseName:'" + data['name'] + "',courseEnrolled:[" + str(data['enroll']) + "], courseRating:" + str(data['rating']) + ",courseFee:" + str(data['fee']) + ",courseTime:" + str(data['time']) + ", courseUpdateDate: [datetime()], courselink: '" + data['link'] + "'})")

In [ ]:
!$SPARK_HOME/bin/pyspark --jars neo4j-connector-apache-spark_2.12-5.0.0_for_spark_3.jar

Python 3.8.15 (default, Oct 12 2022, 19:14:39) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
22/12/07 20:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/

Using Python version 3.8.15 (default, Oct 12 2022 19:14:39)
Spark context Web UI available at http://697de085432f:4040
Spark context available as 'sc' (master = local[*], app id = local-1670445192994).
SparkSession available as 'spark'.
rue)
>>> from pyspark.sql.functions import col,when
>>> course_data = df.select(col('Outcome'),col('Age'))
, "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8").save()
Traceback (most recent call last):
 

In [ ]:
course_data = df2.select(col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'))
#course_data = df.select(col('Outcome'),col('Age'))
course_data.write.format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url","neo4j+s://c4ae1994.databases.neo4j.io") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8") \
  .option("labels", "Course") \
  .option("node.keys","id") \
  .save()

In [ ]:
course_data.write.format("org.neo4j.spark.DataSource").mode("Overwrite").option("url", "neo4j+s://c4ae1994.databases.neo4j.io").option("authentication.type", "basic").option("authentication.basic.username", "neo4j").option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8").option("labels", "Course").option("node.keys","Age").save()

In [ ]:
course_data.show()

+--------------------+------+-------+----+---+--------------------+
|                name|rating| enroll|time|fee|                link|
+--------------------+------+-------+----+---+--------------------+
|    Machine Learning|   4.9|4491564|  61|  0|https://www.cours...|
|Data Visualizatio...|   4.9|  12564|  18|  0|https://www.cours...|
|Spatial Analysis ...|   4.9|  18321|  14|  0|https://www.cours...|
|Python Data Struc...|   4.9| 800203|  19|  0|https://www.cours...|
|Neural Networks a...|   4.9|1013093|  27|  0|https://www.cours...|
|What is Data Scie...|   4.7| 516658|   9|  0|https://www.cours...|
|         Web of Data|   4.1|   4104|  18|  0|https://www.cours...|
|Share Data Throug...|   4.5|  68559|  24|  0|https://www.cours...|
|Visualization for...|   4.4|   6089|  18|  0|https://www.cours...|
|Using Data for Ge...|   4.8|   7270|   5|  0|https://www.cours...|
|Understanding and...|   4.7|  92827|  21|  0|https://www.cours...|
|Tools for Explora...|   0.0|   2271|  18|  0|ht